In [1]:
%cd ../..

/home/eli/AnacondaProjects/epych


In [2]:
%env DASK_LOGGING__DISTRIBUTED=CRITICAL
%env SPYTMPDIR=/mnt/data/tmp_storage
%env SPYLOGLEVEL=CRITICAL
%env SPYPARLOGLEVEL=CRITICAL

env: DASK_LOGGING__DISTRIBUTED=CRITICAL
env: SPYTMPDIR=/mnt/data/tmp_storage
env: SPYLOGLEVEL=CRITICAL
env: SPYPARLOGLEVEL=CRITICAL


In [3]:
import collections
import logging
import numpy as np
import os

import epych
from epych.statistics import alignment

In [4]:
%matplotlib inline

In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
CONDITIONS = ["lonaive", "go_gloexp", "go_seqctl", "lo_gloexp", "lo_rndctl", "igo_seqctl"]

In [7]:
def samplings(cond):
    sampling = epych.recording.Sampling.unpickle("/mnt/data/DRAFT/000253/grandcat_%s" % cond)
    logging.info("Loaded LFPs for condition %s, baseline from (%f, %f)" % (cond, sampling.trials["trial_start"].mean(), sampling.trials["stim1_start"].mean()))
    yield sampling.smap(lambda sig: sig.downsample(32))

In [8]:
def initialize_spectrum(key, signal):
    area = os.path.commonprefix([loc for loc in signal.channels.location])
    return epych.statistics.spectrum.Spectrogram(signal.df, signal.channels, signal.f0, chunk_trials=3, taper="hann")

In [9]:
spectra = {}

In [10]:
for cond in CONDITIONS:
    logging.info("Calculating LFP spectrograms across condition %s" % cond)
    if os.path.exists("/mnt/data/DRAFT/000253/grand_spectrogram_downsample32_%s" % cond):
        continue
    summary = epych.statistic.Summary(alignment.location_prefix, initialize_spectrum)
    summary.calculate(samplings(cond))
    summary.pickle("/mnt/data/DRAFT/000253/grand_spectrogram_downsample32_%s" % cond)
    del summary

INFO:root:Calculating LFP spectrograms across condition lonaive
INFO:root:Loaded LFPs for condition lonaive, baseline from (-3.002524, -2.001683)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [38:45<00:00,  9.30s/it]
INFO:root:Calculating LFP spectrograms across condition go_gloexp
INFO:root:Loaded LFPs for condition go_gloexp, baseline from (-3.002630, -2.001699)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████